In [1]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock, load_json, export_json
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from itertools import count 
from selenium.webdriver.support import expected_conditions as EC

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [3]:
base_url = 'https://untappd.com'
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}
payload = {'country': 'brazil'}

page = requests.get(base_url + '/beer/top_rated', params=payload, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

#print(soup.prettify())

In [4]:
logger.info("Coletando links de cervejas brasileiras do 'top rated'...")
brazilian_beer_links = [a_tag.get('href') for a_tag in soup.select('div.beer-item > a:first-child')]
logger.info("Feito.")
brazilian_beer_links

2023-11-19 17:27:54,146 - INFO - Coletando links de cervejas brasileiras do 'top rated'...
2023-11-19 17:27:54,164 - INFO - Feito.


['/b/spartacus-brewing-iii-three/4283069',
 '/b/spartacus-brewing-v-five/5285289',
 '/b/spartacus-brewing-iv-four/4756418',
 '/b/hocus-pocus-overdrive/1682105',
 '/b/spartacus-brewing-arete-tomo-2/4542383',
 '/b/salvador-brewing-co-cookie-de-chocolate/2908960',
 '/b/spartacus-brewing-etera-tomo-5/4769967',
 '/b/hocus-pocus-elephants-graveyard/2769185',
 '/b/spartacus-brewing-arete-tomo-1/4098457',
 '/b/cervejaria-dogma-rizoma/1536694',
 '/b/hocus-pocus-supersymmetry/1930404',
 '/b/koala-san-brew-cosmic-blur/3452440',
 '/b/trilha-cervejaria-pao-de-mel/3252352',
 '/b/hocus-pocus-rabbit-hole/2321900',
 '/b/spartacus-brewing-fwiw/3905760',
 '/b/spartacus-brewing-barrel-aged-icon-double-chocolate-peanut-butter/5076259',
 '/b/cervejaria-dogma-mosaic-lover/1698497',
 '/b/salvador-brewing-co-engesa-oil-coffee-edition/3036359',
 '/b/mad-dwarf-brewery-md-oloroso-2017/2957221',
 '/b/5-elementos-cervejaria-artesanal-coffee-and-pancake-brunch-stout/2546467',
 '/b/cervejaria-dogma-morning-gringo/265

In [5]:
number_of_brazilian_beer_links = len(brazilian_beer_links)
logger.info(f"Número total de cervejas no 'top rated' do Brasil: {number_of_brazilian_beer_links}")

2023-11-19 17:27:54,201 - INFO - Número total de cervejas no 'top rated' do Brasil: 50


### Acessando as páginas completas das cervejas com Selenium:

Carregando as credenciais de acesso:

In [6]:
dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

Definindo as opções do navegador que o Selenium usará:

In [7]:
options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

Acessando a página completa de cada cerveja do _top rated_ do Brasil clicando no link 'Show more' e minerando os IDs de usuários:

In [8]:
browser = webdriver.Chrome(options=options)

browser.get(base_url + '/login')
browser.find_element(By.NAME,"username").send_keys(EMAIL)
browser.find_element(By.NAME,"password").send_keys(PASSWORD)
time.sleep(30)

interactions = []
comments_by_beer = {}

logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Coleta das infos das interações")
    with tqdm(total=number_of_brazilian_beer_links) as progress_bar:
        progress_bar.set_description("Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil")
        wait = WebDriverWait(browser, 3)
        page_counter = count(1)
        for beer_url in brazilian_beer_links:
                try:
                    comments_by_beer[beer_url] = []
                    browser.get(base_url + beer_url)
                    counter = count(1)

                    # Validando e clicando no Show More para capturar todos os comentários da cerveja atual
                    while True:
                        try:
                            element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[class='more_checkins button yellow track-click more_checkins_logged']")))
                            browser.execute_script("arguments[0].click();", element)
                            time.sleep(1.7)
                            logger.info(f"Link 'Show more' da {str(page_counter)[6:-1]}ª página clicado pela {next(counter)}ª vez")
                        except Exception as e:
                            print(e)
                            logger.info('Fim da página alcançado!')
                            next(page_counter)
                            break
                    
                    full_page = browser.page_source
                    soup = BeautifulSoup(full_page, 'html.parser')
                    comments = soup.select('div.checkin-comment > p.comment-text')
                    for comment in comments:
                        user = comment.parent.parent.select('p.text > a.user')[0]['href']
                        interaction = {'user': user, 'comment': comment.text}
                        print('user : ', user)
                        print('comment : ', comment)
                        comments_by_beer[beer_url].append(interaction)
                        interactions.append(interaction)

                    progress_bar.update(1)
                except Exception as e:
                    print(f"Erro ao buscar comentários para a cerveja: {beer_url}")
                    print(e)
                    progress_bar.update(1)
                    continue
        iteration_time.stop_watch()
    logger.info(f"Operação concluída! O total de comentários coletados foram {len(interactions)}.")

path = '../data/'
logger.info('Exportando a lista de comentários coletados...')
comments_by_beer_file_name = 'comments_by_beer.json'
export_json(comments_by_beer, path + comments_by_beer_file_name)
logger.info('Exportação concluída!')

2023-11-19 17:28:27,250 - INFO - Iniciando uma sessão...
2023-11-19 17:28:27,251 - INFO - A iteração começará agora...
Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil:   0%|          | 0/50 [00:00<?, ?it/s]2023-11-19 17:28:31,007 - INFO - Link 'Show more' da 1ª página clicado pela 1ª vez
2023-11-19 17:28:32,753 - INFO - Link 'Show more' da 1ª página clicado pela 2ª vez
2023-11-19 17:28:34,500 - INFO - Link 'Show more' da 1ª página clicado pela 3ª vez
2023-11-19 17:28:36,250 - INFO - Link 'Show more' da 1ª página clicado pela 4ª vez
2023-11-19 17:28:38,045 - INFO - Link 'Show more' da 1ª página clicado pela 5ª vez
2023-11-19 17:28:39,803 - INFO - Link 'Show more' da 1ª página clicado pela 6ª vez
2023-11-19 17:28:41,577 - INFO - Link 'Show more' da 1ª página clicado pela 7ª vez
2023-11-19 17:28:43,349 - INFO - Link 'Show more' da 1ª página clicado pela 8ª vez
2023-11-19 17:28:45,139 - INFO - Link 'Show more' da 1ª página clicado pela 9ª vez
2023-11-19 17:28:46,934

Message: 



Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil:   2%|▏         | 1/50 [00:27<22:36, 27.68s/it]

user :  /user/BurningHorizons
comment :  <p class="comment-text" id="translate_1313868120">
LCI PITFEST </p>
user :  /user/EnzoSago
comment :  <p class="comment-text" id="translate_1311365938">
Tjs aussi cool. Les adjuncts sont plus posés. </p>
user :  /user/MarekSlawecki
comment :  <p class="comment-text" id="translate_1290932860">
Masterpiece </p>
user :  /user/NoTimeLeft
comment :  <p class="comment-text" id="translate_1283714722">
太他妈甜了....兑着喝还算可以 </p>
user :  /user/qwegkm
comment :  <p class="comment-text" id="translate_1283713857">
甜度爆高，底子还是酱，相比four的甜度增味种类多，three的甜度更高但单一，略微显出底子的酱味，漂亮。4.7 </p>
user :  /user/RodolphoMaximo
comment :  <p class="comment-text" id="translate_1279989323">
É ainda deu bom brejas do armário q em mudança achamos </p>
user :  /user/LeeHomeRain
comment :  <p class="comment-text" id="translate_1278710386">
巴西第一酒厂斯巴达克斯2021年的三周年世涛，目前喝过最好喝的甜品世涛！第一口惊艳！
在两款波旁威士忌桶陈酿，倒入杯中酒体中等不稠，第一口就像是吃冰激凌一样，刚入口的甜抓住你，酒体超级顺滑，不自主的滑下喉咙，在喉咙处开始回口给你烤椰子、肉桂、咖啡、黑巧等等丰富味道体验，特别的惊喜，尤其是你并没太在意的时候，超

2023-11-19 17:28:58,651 - INFO - Link 'Show more' da 2ª página clicado pela 1ª vez
2023-11-19 17:29:00,384 - INFO - Link 'Show more' da 2ª página clicado pela 2ª vez
2023-11-19 17:29:02,118 - INFO - Link 'Show more' da 2ª página clicado pela 3ª vez
2023-11-19 17:29:03,861 - INFO - Link 'Show more' da 2ª página clicado pela 4ª vez
2023-11-19 17:29:05,606 - INFO - Link 'Show more' da 2ª página clicado pela 5ª vez
2023-11-19 17:29:07,364 - INFO - Link 'Show more' da 2ª página clicado pela 6ª vez
2023-11-19 17:29:09,108 - INFO - Link 'Show more' da 2ª página clicado pela 7ª vez
2023-11-19 17:29:10,849 - INFO - Link 'Show more' da 2ª página clicado pela 8ª vez
2023-11-19 17:29:12,606 - INFO - Link 'Show more' da 2ª página clicado pela 9ª vez
2023-11-19 17:29:14,348 - INFO - Link 'Show more' da 2ª página clicado pela 10ª vez
2023-11-19 17:29:16,105 - INFO - Link 'Show more' da 2ª página clicado pela 11ª vez
2023-11-19 17:29:19,276 - INFO - Fim da página alcançado!


...

user :  /user/hestelotto
comment :  <p class="comment-text" id="translate_1331963200">
Uma das melhores da vida! Nunca senti algo assim! Valeu demais Matheus G. !!! </p>
user :  /user/CERVEJAAMARGA
comment :  <p class="comment-text" id="translate_1330158167">
Alucinante </p>
user :  /user/jakeline_ferreira
comment :  <p class="comment-text" id="translate_1328505395">
Minha favorita do festival, socorro que trem bao! </p>
user :  /user/Degust_cerva
comment :  <p class="comment-text" id="translate_1327671814">
Aromática, macia na boca, em aroma e sabor aparecem cereja, biscoito, chocolate e leve café. LCI 26/10. </p>
user :  /user/gleison_gomes
comment :  <p class="comment-text" id="translate_1327616749">
Late check-in.
BBG23 </p>
user :  /user/Betinhojr
comment :  <p class="comment-text" id="translate_1327161818">
									📸Marcos K.									</p>
user :  /user/WesleyKnight
comment :  <p class="comment-text" id="translate_1326932017">
									Brasília Beer Geek									</p>
user :  /use

2023-11-19 17:50:08,155 - INFO - Link 'Show more' da 50ª página clicado pela 1ª vez
2023-11-19 17:50:09,908 - INFO - Link 'Show more' da 50ª página clicado pela 2ª vez
2023-11-19 17:50:11,654 - INFO - Link 'Show more' da 50ª página clicado pela 3ª vez
2023-11-19 17:50:13,421 - INFO - Link 'Show more' da 50ª página clicado pela 4ª vez
2023-11-19 17:50:15,223 - INFO - Link 'Show more' da 50ª página clicado pela 5ª vez
2023-11-19 17:50:16,973 - INFO - Link 'Show more' da 50ª página clicado pela 6ª vez
2023-11-19 17:50:18,723 - INFO - Link 'Show more' da 50ª página clicado pela 7ª vez
2023-11-19 17:50:20,485 - INFO - Link 'Show more' da 50ª página clicado pela 8ª vez
2023-11-19 17:50:22,230 - INFO - Link 'Show more' da 50ª página clicado pela 9ª vez
2023-11-19 17:50:23,976 - INFO - Link 'Show more' da 50ª página clicado pela 10ª vez
2023-11-19 17:50:25,768 - INFO - Link 'Show more' da 50ª página clicado pela 11ª vez
2023-11-19 17:50:28,885 - INFO - Fim da página alcançado!


Message: 



Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil: 100%|██████████| 50/50 [22:02<00:00, 26.30s/it]2023-11-19 17:50:29,809 - INFO - Coleta das infos das interações levou 22m02s para ser executado.
Coletando os comentários das top 50 cervejas melhor avaliadas do Brasil: 100%|██████████| 50/50 [22:02<00:00, 26.45s/it]
2023-11-19 17:50:29,810 - INFO - Operação concluída! O total de comentários coletados foram 3006.
2023-11-19 17:50:29,811 - INFO - Exportando a lista de comentários coletados...
2023-11-19 17:50:29,834 - INFO - Exportação concluída!


user :  /user/fernando_barbosa_candido_8933
comment :  <p class="comment-text" id="translate_1216837950">
Checkin de quando ainda usava fraldas </p>
user :  /user/thebennich
comment :  <p class="comment-text" id="translate_1237564642">
Muda de sabor a cada "safra". Tá na hora de mudar de nome, não é? </p>
user :  /user/gabrielflc
comment :  <p class="comment-text" id="translate_1083808483">
REVISITANDO ESSE CLÁSSICO Bruno M. </p>
user :  /user/Sir_Veja
comment :  <p class="comment-text" id="translate_821778067">
Lci </p>
user :  /user/adolfofeldmanndeschnaid
comment :  <p class="comment-text" id="translate_744467000">
									Néctar dia Deuses!!									</p>
user :  /user/JucaVieira
comment :  <p class="comment-text" id="translate_703848102">
									#latecheckin									</p>
user :  /user/AraujoLC
comment :  <p class="comment-text" id="translate_590368536">
									Como faz para arrumar um patrocínio?									</p>
user :  /user/dollyhx
comment :  <p class="comment-text" id="transla